In [1]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision
from matplotlib import pyplot as plt
import numpy as np
from torch import optim
from torch import nn
import os
from PIL import Image

In [2]:
train_path = r"D:\dataSource\ImgNet\train"
test_path = r"D:\dataSource\ImgNet\test"

In [3]:
from torchvision.transforms import ToTensor  # 用于把图片转化为张量
import numpy as np  # 用于将张量转化为数组，进行除法
from torchvision.datasets import ImageFolder  # 用于导入图片数据集

means = [0, 0, 0]
std = [0, 0, 0]  # 初始化均值和方差
transform = ToTensor()  # 可将图片类型转化为张量，并把0~255的像素值缩小到0~1之间
dataset = ImageFolder(train_path, transform=transform)  # 导入数据集的图片，并且转化为张量
num_imgs = len(dataset)  # 获取数据集的图片数量
for img, a in dataset:  # 遍历数据集的张量和标签
    for i in range(3):  # 遍历图片的RGB三通道
        # 计算每一个通道的均值和标准差
        means[i] += img[i, :, :].mean()
        std[i] += img[i, :, :].std()
mean = np.array(means) / num_imgs
std = np.array(std) / num_imgs  # 要使数据集归一化，均值和方差需除以总图片数量
print(mean, std)  # 打印出结果

[0.38099068 0.4367647  0.4106545 ] [0.19743624 0.18913363 0.1877521 ]


In [4]:
BATCH_SIZE = 400
transforms_train = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]
)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

transforms_test = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]
)

In [5]:
class loadImg(Dataset):
    def __init__(self, dataDir, transform=None, imgTypes=None, howConvert="RGB"):
        if transform:
            self.transform = transform
        if not imgTypes:
            self.imgTypes = ["jpg", "png",'jpeg']
        self.dataDir = dataDir
        self.howConvert = howConvert
        self.labelsDict = self.getLabelsDict()
        self.dataInfo = self.loadFile()

    
    def getLabelsDict(self):
        for _, dirs, _ in os.walk(self.dataDir):
            return {label: key for key, label in enumerate(dirs)}

    def loadFile(self):
        allFile = []
        for root, dirs, _ in os.walk(self.dataDir):
            for dir in dirs:
                files = os.listdir(os.path.join(root, dir))
                files = [
                    (os.path.join(root,dir,imgPath), self.labelsDict[dir])
                    for imgPath in files
                    if str(imgPath).split('.')[-1].lower() in self.imgTypes
                ]
                allFile.extend(files)
        return allFile

    def __len__(self):
        return len(self.loadFile())

    def __getitem__(self, index):
        imgPath, label = self.dataInfo[index]
        img = Image.open(imgPath).convert(self.howConvert)
        if self.transform:
            img = self.transform(imgPath)   
        return img, label

In [6]:
# data_train = loadImg(train_path, transform=transforms_train)
# data_train_loader = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
# data_test = loadImg(test_path, transform=transforms_test)
# data_test_loader = DataLoader(data_test, batch_size=BATCH_SIZE)

In [7]:
data_train = ImageFolder(train_path, transform=transforms_train)
data_train_loader = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
data_test = ImageFolder(test_path, transform=transforms_test)
data_test_loader = DataLoader(data_test, batch_size=BATCH_SIZE)

In [8]:
len(data_test_loader.dataset), len(data_train_loader.dataset)

(1200, 4000)

In [9]:
from torchvision.models import resnet50

net = resnet50()

In [10]:
def train(net,  lossFn, optimzer, dataloader,device):
    size = len(dataloader.dataset)  # 整体数据的大小
    net.train()
    for batch, X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimzer.zero_grad()
        pred = net(X)
        """
        pred可能存在的预处理
        1、pred=(pred.argmax(dim=1) == y).type(torch.float).sum().item()
        2、pred=torch.where(pred>0.5,1.,0.)
        3、pred.requires_grad=True # 设置可导
        """
        loss = lossFn(pred, y)
        loss.backward()
        optimzer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(net, lossFn,dataloader, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    net.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = net(X)  # pred此处可能的操作同上
            test_loss += lossFn(pred, y).item()
            # 获取pred每列的最大值的index与y比较
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

In [11]:
def train(net, loss_fn, optimizer, train_data, device):
    net.to(device)
    net.train()
    size = len(train_data.dataset)
    for batch, (X, y )in enumerate(train_data):
        print(len(X))
        X=X.to(device)
        y=y.to(device)
        optimizer.zero_grad()
        pred = net(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        if ((batch + 1) % 10) == 0:
            loss, current = loss.item(), (batch + 1) * len(y)
            print(f"Train Info:\n Loss:{loss:>7f}  {current:>4d}/{size:4>d}")


def test(net, loss_fn, test_data, device):
    net.to(device)
    net.eval()
    size = len(test_data.dataset)
    batch_size = len(test_data)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in test_data:
            X=X.to(device)
            y=y.to(device)
            pred = net(X)
            test_loss += loss_fn(pred, y).item()
            correct += y.eq(pred.argmax(1).view_as(y)).sum().item()
    test_loss /=batch_size
    correct/=size
    print(f"Test Info:\n Avg Loss:{test_loss:>7f}  Accuracy:{correct*100:>0.1f}%")

In [12]:
def train(model, loss_fn, optimizer, dataloader, device):
    size = len(dataloader.dataset)
    model.train()
    model.to(device)
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss +=loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 2 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print(f"Train Error: \n Accuracy: {(100*correct/size):>0.1f}%, Avg loss: {train_loss:>8f} \n")
            
def test(model, loss_fn, dataloader, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.to(device)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [13]:
from torchvision.models import resnet18

net = resnet18(pretrained=True)
# fc input = resnet34.fc.in_features
#for param in net.parameters():
#    param.requires_grad = False
net

d:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
net.fc=nn.Linear(in_features=net.fc.in_features, out_features=4, bias=True)
nn.init.xavier_normal_(net.fc.weight)

Parameter containing:
tensor([[ 0.0514, -0.0582, -0.0373,  ...,  0.0008, -0.1011,  0.0985],
        [ 0.0705, -0.1134,  0.0513,  ...,  0.0147,  0.0203,  0.0242],
        [-0.0190,  0.0951,  0.0211,  ..., -0.0715,  0.0466, -0.0062],
        [-0.0064, -0.0596, -0.0172,  ..., -0.0289, -0.1096,  0.0798]],
       requires_grad=True)

In [15]:
from torchkeras import summary
for features,labels in data_train_loader:
    break 
summary(net.cuda(),features.cuda())

--------------------------------------------------------------------------
Layer (type)                            Output Shape              Param #
Conv2d-1                          [-1, 64, 112, 112]                9,408
BatchNorm2d-2                     [-1, 64, 112, 112]                  128
ReLU-3                            [-1, 64, 112, 112]                    0
MaxPool2d-4                         [-1, 64, 56, 56]                    0
Conv2d-5                            [-1, 64, 56, 56]               36,864
BatchNorm2d-6                       [-1, 64, 56, 56]                  128
ReLU-7                              [-1, 64, 56, 56]                    0
Conv2d-8                            [-1, 64, 56, 56]               36,864
BatchNorm2d-9                       [-1, 64, 56, 56]                  128
ReLU-10                             [-1, 64, 56, 56]                    0
Conv2d-11                           [-1, 64, 56, 56]               36,864
BatchNorm2d-12                      [

In [16]:
for name , param in net.named_parameters():
    print(name)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.weight
layer2.1.bn2.weight
layer2.1.bn2.bias
layer3.0.conv1.weight
layer3.0.bn1.weight
layer3.0.bn1.bias
layer3.0.conv2.weight
layer3.0.bn2.weight
layer3.0.bn2.bias
layer3.0.downsample.0.weight
layer3.0.downsample.1.weight
layer3.0.downsample.1.bias
layer3.1.conv1.weight
layer3.1.bn1.weight
layer3.1.bn1.bias
layer3.1.conv2.weight
layer3.1.bn2.weight
layer3.1.bn2.bias
layer4.0.conv1.weight
layer4.0.bn1.weight
layer4.0.bn1.bias
layer4.0.conv2.we

In [17]:
lr = 0.001
params_1x = [
    param
    for name, param in net.named_parameters()
    if name not in ("fc.weight", "fc.bias")
]
trainer = torch.optim.SGD(
    [{"params": params_1x}, {"params": net.fc.parameters(), "lr": lr * 10}],
    lr=lr,
    weight_decay=0.001,
)

In [18]:
import torch
import random
import numpy as np

# 设置随机种子
seed = 42


np.random.seed(seed)# 设置 NumPy 中的随机种子
random.seed(seed) #设置 Python 标准库中的随机种子，以确保其他 Python 函数中使用的随机数也是可复现的。


torch.manual_seed(seed)
torch.cuda.manual_seed(seed) #设置 PyTorch 在 CUDA 环境下的随机种子，以确保 CUDA 计算的结果是可复现的。
torch.cuda.manual_seed_all(seed)  # 如果使用多个GPU，此命令将确保所有的 GPU 使用相同的随机种子。
torch.backends.cudnn.deterministic = True # 确保在使用 cuDNN 加速时结果可复现，但可能会降低性能。
torch.backends.cudnn.benchmark = False #禁用 cuDNN 的自动寻找最适合当前配置的高效算法的功能，以确保结果的一致性。


In [20]:
epochs = 5
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(net.parameters(),lr = 1e-4)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(net,loss_fn,trainer,data_train_loader,DEVICE)
    test(net,loss_fn,data_train_loader,DEVICE)
print("Done!")

Epoch 1
-------------------------------
loss: 0.257167  [  400/ 4000]
loss: 0.232451  [ 1200/ 4000]
loss: 0.224507  [ 2000/ 4000]
loss: 0.233458  [ 2800/ 4000]
loss: 0.202608  [ 3600/ 4000]
Train Error: 
 Accuracy: 93.4%, Avg loss: 2.352909 

Test Error: 
 Accuracy: 94.1%, Avg loss: 0.214489 

Epoch 2
-------------------------------
loss: 0.225118  [  400/ 4000]
loss: 0.179021  [ 1200/ 4000]
loss: 0.212692  [ 2000/ 4000]
loss: 0.208570  [ 2800/ 4000]
loss: 0.192938  [ 3600/ 4000]
Train Error: 
 Accuracy: 94.0%, Avg loss: 2.113919 

Test Error: 
 Accuracy: 94.7%, Avg loss: 0.195570 

Epoch 3
-------------------------------
loss: 0.200472  [  400/ 4000]
loss: 0.236070  [ 1200/ 4000]
loss: 0.182307  [ 2000/ 4000]
loss: 0.181441  [ 2800/ 4000]
loss: 0.187837  [ 3600/ 4000]
Train Error: 
 Accuracy: 94.7%, Avg loss: 1.945870 

Test Error: 
 Accuracy: 95.0%, Avg loss: 0.180235 

Epoch 4
-------------------------------
loss: 0.166694  [  400/ 4000]
loss: 0.198951  [ 1200/ 4000]
loss: 0.181941 